In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report, precision_score,recall_score,f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
style.use('ggplot')

df_2 = pd.read_excel('PL_2018-2019.xlsx')
df = pd.read_excel('PL_2019-2020.xlsx')
df_3=df.append(df_2)
df_3=df_3.reset_index(drop=True)
df,df_2,df_3

C:\Users\91720\.conda\envs\env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(       SEASON_CD SEASON_NAME  PLOT_CLUSTER_CD  CLUSTER_NAME  PLOT_VILLAGE_CD  \
 0             25   2019-2020                1  FACTORY EAST                2   
 1             25   2019-2020                1  FACTORY EAST                2   
 2             25   2019-2020                1  FACTORY EAST                2   
 3             25   2019-2020                1  FACTORY EAST                2   
 4             25   2019-2020                1  FACTORY EAST                2   
 ...          ...         ...              ...           ...              ...   
 53420         25   2019-2020               28       LOKAPUR             1453   
 53421         25   2019-2020               28       LOKAPUR             1453   
 53422         25   2019-2020               28       LOKAPUR             1468   
 53423         25   2019-2020               28       LOKAPUR             1468   
 53424         25   2019-2020               28       LOKAPUR             1468   
 
          VILLAGE_NAME  FA

In [2]:
CV_NAMES=['86032','91010','M 265','SNK-09293','PI 00-1110','CO VSI 08005']
df1=df[(df.CV_NAME.isin (CV_NAMES)) & 
       (((df.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days) > 0) &
       (df.AGE_IN_DAYS>0)].copy()
df1['DISPOSAL_TIME']=((df1.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days)

df1_2=df_2[(df_2.CV_NAME.isin (CV_NAMES)) & 
       (((df_2.DISPOSAL_DATE - datetime.datetime(2017,1,1)).dt.days) > 0) &
       (df_2.AGE_IN_DAYS>0)].copy()
df1_2['DISPOSAL_TIME']=((df1_2.DISPOSAL_DATE - datetime.datetime(2017,1,1)).dt.days)
df1,df1_2

(       SEASON_CD SEASON_NAME  PLOT_CLUSTER_CD  CLUSTER_NAME  PLOT_VILLAGE_CD  \
 0             25   2019-2020                1  FACTORY EAST                2   
 1             25   2019-2020                1  FACTORY EAST                2   
 2             25   2019-2020                1  FACTORY EAST                2   
 3             25   2019-2020                1  FACTORY EAST                2   
 4             25   2019-2020                1  FACTORY EAST                2   
 ...          ...         ...              ...           ...              ...   
 53420         25   2019-2020               28       LOKAPUR             1453   
 53421         25   2019-2020               28       LOKAPUR             1453   
 53422         25   2019-2020               28       LOKAPUR             1468   
 53423         25   2019-2020               28       LOKAPUR             1468   
 53424         25   2019-2020               28       LOKAPUR             1468   
 
          VILLAGE_NAME  FA

In [3]:
df2=df1.sort_values('DISPOSAL_TIME').copy()
df2['DISPO_CURR']=0
for i in CV_NAMES:
    df2['DISPO_'+ i]= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == i),other=0)
    df2['DISPO_' + i]=df2['DISPO_' + i].cumsum()
    df2['DISPO_CURR']= df2['DISPO_' + i].where(cond=(df2.CV_NAME == i),other=df2['DISPO_CURR'])
df2_2=df1_2.sort_values('DISPOSAL_TIME').copy()
df2_2['DISPO_CURR']=0
for i in CV_NAMES:
    df2_2['DISPO_'+ i]= df_2.DISPOSAL_TONNS.where(cond=(df2_2.CV_NAME == i),other=0)
    df2_2['DISPO_' + i]=df2_2['DISPO_' + i].cumsum()
    df2_2['DISPO_CURR']= df2_2['DISPO_' + i].where(cond=(df2_2.CV_NAME == i),other=df2_2['DISPO_CURR'])

df2,df2_2

(       SEASON_CD SEASON_NAME  PLOT_CLUSTER_CD  CLUSTER_NAME  PLOT_VILLAGE_CD  \
 48426         25   2019-2020               20    SHIRAGUPPI              912   
 46277         25   2019-2020               20    SHIRAGUPPI              905   
 46937         25   2019-2020               20    SHIRAGUPPI              907   
 11908         25   2019-2020                2    FACTORY-SW              900   
 39221         25   2019-2020               18       TERADAL              880   
 ...          ...         ...              ...           ...              ...   
 3189          25   2019-2020                1  FACTORY EAST                8   
 12058         25   2019-2020                2    FACTORY-SW              900   
 7946          25   2019-2020                2    FACTORY-SW               16   
 7534          25   2019-2020                2    FACTORY-SW               15   
 7883          25   2019-2020                2    FACTORY-SW               16   
 
          VILLAGE_NAME  FA

In [4]:
one=pd.DataFrame({'a':df2.DISPO_CURR})
two = pd.DataFrame({"a":[0]}).copy()
two=two.append(one)
two=two.reset_index(drop=True)
df3=df2.reset_index(drop=True).copy()
df3['DISPO_CURR2']=two['a']
one=pd.DataFrame({'a':df2_2.DISPO_CURR})
two = pd.DataFrame({"a":[0]}).copy()
two=two.append(one)
two=two.reset_index(drop=True)
df3_2=df2_2.reset_index(drop=True).copy()
df3_2['DISPO_CURR2']=two['a']
df3,df3_2

(       SEASON_CD SEASON_NAME  PLOT_CLUSTER_CD  CLUSTER_NAME  PLOT_VILLAGE_CD  \
 0             25   2019-2020               20    SHIRAGUPPI              912   
 1             25   2019-2020               20    SHIRAGUPPI              905   
 2             25   2019-2020               20    SHIRAGUPPI              907   
 3             25   2019-2020                2    FACTORY-SW              900   
 4             25   2019-2020               18       TERADAL              880   
 ...          ...         ...              ...           ...              ...   
 51991         25   2019-2020                1  FACTORY EAST                8   
 51992         25   2019-2020                2    FACTORY-SW              900   
 51993         25   2019-2020                2    FACTORY-SW               16   
 51994         25   2019-2020                2    FACTORY-SW               15   
 51995         25   2019-2020                2    FACTORY-SW               16   
 
          VILLAGE_NAME  FA

In [5]:
X3=df3[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME","DISPO_CURR2"]].copy()
Y3=df3["DIVERSION_CENTER_NAME"].copy()

for i in Y3.index:
    Y3[i]=(1 if (Y3[i]=="GSM_HARVEST") else 0)
Y3_new=Y3.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X3_new = pd.DataFrame(transformer.fit_transform(X3))

X3_2=df3_2[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME","DISPO_CURR2"]].copy()
Y3_2=df3_2["DIVERSION_CENTER_NAME"].copy()

for i in Y3_2.index:
    Y3_2[i]=(1 if (Y3_2[i]=="GSM_HARVEST") else 0)
Y3_2_new=Y3_2.astype('int')

X3_2_new = pd.DataFrame(transformer.fit_transform(X3_2))

X3_3_new=X3_2_new.append(X3_new)
X3_3_new=X3_3_new.reset_index(drop=True)

Y3_3_new=Y3_2_new.append(Y3_new)
Y3_3_new=Y3_3_new.reset_index(drop=True)

X3_new,Y3_new,X3_2_new,Y3_2_new,X3_3_new,Y3_3_new

(        0    1    2    3    4    5    6    7    8    9   ...   23   24   25  \
 0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 3      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 51991  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 51992  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 51993  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 51994  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 51995  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 
         26   27   28    29     30    

In [24]:
X3train,X3test,Y3train,Y3test =train_test_split(X3_new,Y3_new,test_size=0.2)
X3_2train,X3_2test,Y3_2train,Y3_2test =train_test_split(X3_2_new,Y3_2_new,test_size=0.2)
X3_3train,X3_3test,Y3_3train,Y3_3test =train_test_split(X3_3_new,Y3_3_new,test_size=0.2)
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
def cv_score(model,X,Y):
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    return('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
def score(gs):
    gs.fit(X3train,Y3train)
    Y3preds=gs.predict(X3test)
    return(pd.DataFrame(classification_report(Y3test,Y3preds, output_dict=True)).transpose(),
           confusion_matrix(Y3test,Y3preds))

In [7]:
hgbc = HistGradientBoostingClassifier()
xgboost = XGBClassifier(use_label_encoder=False)
catboost = CatBoostClassifier()
lgbm = LGBMClassifier()
rf = RandomForestClassifier()
kn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=4000)
rc = RidgeClassifier()
svc = SVC()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()

In [13]:
print('data from 2019-2020:')
print('CATBoost:',cv_score(catboost,X3_new,Y3_new))
print('Random forest:',cv_score(rf,X3_new,Y3_new))
print('XGBoost:',cv_score(xgboost,X3_new,Y3_new))
print('LightGradientBoost:',cv_score(lgbm,X3_new,Y3_new))
print('HistGradientBoost:',cv_score(hgbc,X3_new,Y3_new))
print('')
print('data from 2018-2019')
print('CATBoost:',cv_score(catboost,X3_2_new,Y3_2_new))
print('Random forest:',cv_score(rf,X3_2_new,Y3_2_new))
print('XGBoost:',cv_score(xgboost,X3_2_new,Y3_2_new))
print('LightGradientBoost:',cv_score(lgbm,X3_2_new,Y3_2_new))
print('HistGradientBoost:',cv_score(hgbc,X3_2_new,Y3_2_new))
print('')
print('data from 2018-2020')
print('CATBoost:',cv_score(catboost,X3_3_new,Y3_3_new))
print('Random forest:',cv_score(rf,X3_3_new,Y3_3_new))
print('XGBoost:',cv_score(xgboost,X3_3_new,Y3_3_new))
print('LightGradientBoost:',cv_score(lgbm,X3_3_new,Y3_3_new))
print('HistGradientBoost:',cv_score(hgbc,X3_3_new,Y3_3_new))

data from 2019-2020:
CATBoost: Accuracy: 0.909 (0.003)
Random forest: Accuracy: 0.903 (0.005)
XGBoost: Accuracy: 0.909 (0.004)
LightGradientBoost: Accuracy: 0.905 (0.004)
HistGradientBoost: Accuracy: 0.905 (0.004)

data from 2018-2019
CATBoost: Accuracy: 0.901 (0.004)
Random forest: Accuracy: 0.896 (0.003)
XGBoost: Accuracy: 0.899 (0.004)
LightGradientBoost: Accuracy: 0.896 (0.004)
HistGradientBoost: Accuracy: 0.896 (0.004)

data from 2018-2020
CATBoost: Accuracy: 0.902 (0.002)
Random forest: Accuracy: 0.898 (0.003)
XGBoost: Accuracy: 0.900 (0.001)
LightGradientBoost: Accuracy: 0.894 (0.002)
HistGradientBoost: Accuracy: 0.895 (0.002)


In [25]:
print('CATBoost:',score(catboost),sep='\n')
print('')
print('Random forest:',score(rf),sep='\n')
print('')
print('XGBoost:',score(xgboost),sep='\n')
print('')
print('LightGradientBoost:',score(lgbm),sep='\n')
print('')
print('HistGradientBoost:',score(hgbc),sep='\n')

Learning rate set to 0.050614
0:	learn: 0.6280386	total: 20.2ms	remaining: 20.2s
1:	learn: 0.5703855	total: 36.7ms	remaining: 18.3s
2:	learn: 0.5255013	total: 51.9ms	remaining: 17.2s
3:	learn: 0.4919909	total: 66.4ms	remaining: 16.5s
4:	learn: 0.4641208	total: 81.4ms	remaining: 16.2s
5:	learn: 0.4385643	total: 95.4ms	remaining: 15.8s
6:	learn: 0.4204991	total: 110ms	remaining: 15.6s
7:	learn: 0.4063916	total: 123ms	remaining: 15.3s
8:	learn: 0.3914111	total: 137ms	remaining: 15s
9:	learn: 0.3798202	total: 151ms	remaining: 14.9s
10:	learn: 0.3712679	total: 165ms	remaining: 14.9s
11:	learn: 0.3606233	total: 181ms	remaining: 14.9s
12:	learn: 0.3523957	total: 197ms	remaining: 15s
13:	learn: 0.3478232	total: 215ms	remaining: 15.2s
14:	learn: 0.3427820	total: 229ms	remaining: 15s
15:	learn: 0.3369812	total: 242ms	remaining: 14.9s
16:	learn: 0.3323549	total: 254ms	remaining: 14.7s
17:	learn: 0.3288956	total: 267ms	remaining: 14.6s
18:	learn: 0.3251276	total: 280ms	remaining: 14.5s
19:	learn: 

174:	learn: 0.2405506	total: 2.34s	remaining: 11s
175:	learn: 0.2403934	total: 2.36s	remaining: 11s
176:	learn: 0.2401369	total: 2.37s	remaining: 11s
177:	learn: 0.2398675	total: 2.38s	remaining: 11s
178:	learn: 0.2397670	total: 2.4s	remaining: 11s
179:	learn: 0.2396088	total: 2.41s	remaining: 11s
180:	learn: 0.2394209	total: 2.42s	remaining: 11s
181:	learn: 0.2392854	total: 2.43s	remaining: 10.9s
182:	learn: 0.2391904	total: 2.45s	remaining: 10.9s
183:	learn: 0.2390197	total: 2.46s	remaining: 10.9s
184:	learn: 0.2387860	total: 2.47s	remaining: 10.9s
185:	learn: 0.2386427	total: 2.48s	remaining: 10.9s
186:	learn: 0.2384310	total: 2.5s	remaining: 10.9s
187:	learn: 0.2382488	total: 2.51s	remaining: 10.8s
188:	learn: 0.2379944	total: 2.53s	remaining: 10.9s
189:	learn: 0.2378292	total: 2.55s	remaining: 10.9s
190:	learn: 0.2377112	total: 2.57s	remaining: 10.9s
191:	learn: 0.2375038	total: 2.58s	remaining: 10.9s
192:	learn: 0.2373883	total: 2.59s	remaining: 10.8s
193:	learn: 0.2372901	total:

346:	learn: 0.2195133	total: 4.52s	remaining: 8.5s
347:	learn: 0.2194297	total: 4.53s	remaining: 8.48s
348:	learn: 0.2193393	total: 4.54s	remaining: 8.47s
349:	learn: 0.2192364	total: 4.55s	remaining: 8.45s
350:	learn: 0.2190989	total: 4.56s	remaining: 8.44s
351:	learn: 0.2190095	total: 4.57s	remaining: 8.42s
352:	learn: 0.2189498	total: 4.58s	remaining: 8.4s
353:	learn: 0.2188727	total: 4.6s	remaining: 8.39s
354:	learn: 0.2187866	total: 4.61s	remaining: 8.37s
355:	learn: 0.2186892	total: 4.62s	remaining: 8.35s
356:	learn: 0.2186117	total: 4.63s	remaining: 8.34s
357:	learn: 0.2185365	total: 4.64s	remaining: 8.32s
358:	learn: 0.2185063	total: 4.65s	remaining: 8.3s
359:	learn: 0.2184225	total: 4.66s	remaining: 8.29s
360:	learn: 0.2183311	total: 4.67s	remaining: 8.27s
361:	learn: 0.2182523	total: 4.68s	remaining: 8.25s
362:	learn: 0.2181594	total: 4.69s	remaining: 8.23s
363:	learn: 0.2180656	total: 4.71s	remaining: 8.22s
364:	learn: 0.2179987	total: 4.72s	remaining: 8.21s
365:	learn: 0.21

508:	learn: 0.2081418	total: 6.51s	remaining: 6.28s
509:	learn: 0.2080633	total: 6.52s	remaining: 6.26s
510:	learn: 0.2080305	total: 6.53s	remaining: 6.25s
511:	learn: 0.2079751	total: 6.54s	remaining: 6.24s
512:	learn: 0.2079084	total: 6.55s	remaining: 6.22s
513:	learn: 0.2078374	total: 6.57s	remaining: 6.21s
514:	learn: 0.2077863	total: 6.58s	remaining: 6.19s
515:	learn: 0.2077371	total: 6.59s	remaining: 6.18s
516:	learn: 0.2076711	total: 6.6s	remaining: 6.17s
517:	learn: 0.2075749	total: 6.61s	remaining: 6.15s
518:	learn: 0.2075568	total: 6.62s	remaining: 6.14s
519:	learn: 0.2074930	total: 6.63s	remaining: 6.12s
520:	learn: 0.2074247	total: 6.65s	remaining: 6.11s
521:	learn: 0.2073718	total: 6.67s	remaining: 6.11s
522:	learn: 0.2072704	total: 6.68s	remaining: 6.09s
523:	learn: 0.2072055	total: 6.69s	remaining: 6.08s
524:	learn: 0.2071462	total: 6.71s	remaining: 6.07s
525:	learn: 0.2070788	total: 6.72s	remaining: 6.05s
526:	learn: 0.2070424	total: 6.73s	remaining: 6.04s
527:	learn: 0

667:	learn: 0.1998460	total: 8.43s	remaining: 4.19s
668:	learn: 0.1997945	total: 8.44s	remaining: 4.18s
669:	learn: 0.1997280	total: 8.46s	remaining: 4.17s
670:	learn: 0.1996938	total: 8.47s	remaining: 4.15s
671:	learn: 0.1996172	total: 8.48s	remaining: 4.14s
672:	learn: 0.1995778	total: 8.49s	remaining: 4.13s
673:	learn: 0.1995282	total: 8.5s	remaining: 4.11s
674:	learn: 0.1994826	total: 8.52s	remaining: 4.1s
675:	learn: 0.1994498	total: 8.53s	remaining: 4.09s
676:	learn: 0.1993931	total: 8.54s	remaining: 4.07s
677:	learn: 0.1993351	total: 8.55s	remaining: 4.06s
678:	learn: 0.1993064	total: 8.56s	remaining: 4.05s
679:	learn: 0.1992553	total: 8.57s	remaining: 4.03s
680:	learn: 0.1991836	total: 8.58s	remaining: 4.02s
681:	learn: 0.1991053	total: 8.59s	remaining: 4.01s
682:	learn: 0.1990641	total: 8.61s	remaining: 3.99s
683:	learn: 0.1990076	total: 8.62s	remaining: 3.98s
684:	learn: 0.1989777	total: 8.63s	remaining: 3.97s
685:	learn: 0.1989039	total: 8.64s	remaining: 3.96s
686:	learn: 0.

839:	learn: 0.1923373	total: 10.6s	remaining: 2.02s
840:	learn: 0.1922997	total: 10.6s	remaining: 2s
841:	learn: 0.1922406	total: 10.6s	remaining: 1.99s
842:	learn: 0.1922082	total: 10.6s	remaining: 1.98s
843:	learn: 0.1921383	total: 10.6s	remaining: 1.97s
844:	learn: 0.1921216	total: 10.7s	remaining: 1.96s
845:	learn: 0.1920844	total: 10.7s	remaining: 1.94s
846:	learn: 0.1920656	total: 10.7s	remaining: 1.93s
847:	learn: 0.1920459	total: 10.7s	remaining: 1.92s
848:	learn: 0.1919855	total: 10.7s	remaining: 1.91s
849:	learn: 0.1919464	total: 10.7s	remaining: 1.89s
850:	learn: 0.1919094	total: 10.7s	remaining: 1.88s
851:	learn: 0.1918412	total: 10.7s	remaining: 1.87s
852:	learn: 0.1918071	total: 10.8s	remaining: 1.85s
853:	learn: 0.1917659	total: 10.8s	remaining: 1.84s
854:	learn: 0.1916973	total: 10.8s	remaining: 1.83s
855:	learn: 0.1916504	total: 10.8s	remaining: 1.81s
856:	learn: 0.1916150	total: 10.8s	remaining: 1.8s
857:	learn: 0.1915616	total: 10.8s	remaining: 1.79s
858:	learn: 0.19

998:	learn: 0.1860777	total: 12.6s	remaining: 12.6ms
999:	learn: 0.1860353	total: 12.6s	remaining: 0us
CATBoost:
(              precision    recall  f1-score       support
0              0.948625  0.894270  0.920646   5864.000000
1              0.872742  0.937390  0.903912   4536.000000
accuracy       0.913077  0.913077  0.913077      0.913077
macro avg      0.910684  0.915830  0.912279  10400.000000
weighted avg   0.915529  0.913077  0.913347  10400.000000, array([[5244,  620],
       [ 284, 4252]], dtype=int64))

Random forest:
(              precision    recall  f1-score       support
0              0.936360  0.893247  0.914296   5864.000000
1              0.869746  0.921517  0.894883   4536.000000
accuracy       0.905577  0.905577  0.905577      0.905577
macro avg      0.903053  0.907382  0.904590  10400.000000
weighted avg   0.907306  0.905577  0.905829  10400.000000, array([[5238,  626],
       [ 356, 4180]], dtype=int64))

[19:08:54] WARNING: C:/Users/Administrator/workspace/xgb